Example of looping through all comments, and storing info in a dataframe

In [1]:
#Grab packages

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
#Connect to webpage

options = webdriver.ChromeOptions()
#options.add_argument('--headless')


options.binary_location = "/usr/bin/google-chrome"
chrome_driver_binary = "/usr/local/bin/chromedriver"
#options.binary_location = "C:/Program Files (x86)/Google/Chrome/Application/chrome.exe"
#chrome_driver_binary = "C:/Users/Alek/chromedriver"
driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)



/home/ravisolter/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [3]:
# Logging in
driver.get("https://auth.voxmedia.com/login?return_to=https://www.blackwhitereadallover.com/")

element = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.CLASS_NAME, "p-text-input"))
    )

username = driver.find_element_by_xpath('//*[@id="login[username]"]')
username.clear()
username.send_keys("rsolter")


password = driver.find_element_by_xpath('//*[@id="login[password]"]')
password.clear()
password.send_keys("pass123!")


button = driver.find_element_by_xpath('//*[@id="auth"]/div/form/fieldset[3]/input')
button.click()

In [11]:
# Directing to one example article
element = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.CLASS_NAME, "c-global-header__logo-large"))
    )




In [4]:

### Insert loop here to visit relevant URLs

driver.get("https://www.blackwhitereadallover.com/2020/4/3/21206623/miralem-pjanic-marco-verratti-juventus-paris-saint-germain-2020-serie-a-summer-transfer-rumors#comments")

## Grab article title, author

element = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.CLASS_NAME, "c-comments__list"))
    )

#Find number of comments

n_comments = driver.find_element_by_xpath('//*[@id="comments"]/div[1]/span/span')
n_comments = int(n_comments.text)


In [41]:
#Loop through all comments, taking title, content, and meta data (recs, author, etc)

comments_df = pd.DataFrame()

for i in range(1,n_comments+1):
    
    xpath = '//*[@id="comments"]/div[2]/div['+str(i)+']'
    title = driver.find_element_by_xpath(xpath + '/div[1]')
    body = driver.find_element_by_xpath(xpath + '/div[2]')
    meta = driver.find_element_by_xpath(xpath + '/div[3]')
    #author = driver.find_element_by_xpath(xpath + '/div[3]/span[1]/a[1]')
    #recs = driver.find_element_by_xpath(xpath + '/div[3]/span[2]/button[2]/span[2]')
    
    
    author_xpath = '/html/body/div[2]/main/article/div[5]/div[1]/section/div[2]/div['+str(i)+']/div[3]/span[1]/a[1]'
    author = driver.find_element_by_xpath(author_xpath)

    #recs_xpath = '/html/body/div[2]/main/article/div[5]/div[1]/section/div[2]/div['+str(i)+']/div[3]/span[2]/button[2]/span[2]'
    #recs = driver.find_element_by_xpath(recs_xpath)
    
    

    #author = driver.find_element_by_xpath('/html/body/div[2]/main/article/div[5]/div[1]/section/div[2]/div['+str(i)+']/div[3]/span[1]/a[1]')
    #recs = driver.find_element_by_xpath('/html/body/div[2]/main/article/div[5]/div[1]/section/div[2]/div[2]/div[3]/span[2]/button[2]/span[2]')

    
    author2 = author.text
    #recs2 = recs.text
    

    comment = pd.DataFrame([title.text,body.text,meta.text,author2])
    comments_df = pd.concat([comments_df,comment.T]);

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=55897): Max retries exceeded with url: /session/f1357c63a50314bb00f9bd8500bdc1d4/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7a40c67c3590>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [42]:
comments_df.columns = ['Title','Body','Meta','Author']

comments_df.head()

ValueError: Length mismatch: Expected axis has 0 elements, new values have 4 elements

In [40]:
driver.quit()